<a href="https://colab.research.google.com/github/Deep-of-Machine/AI_Academy/blob/main/5_2_torch%EA%B8%B0%EC%B4%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GPU 연결 확인
런타임 - 런타임 유형 변경 - GPU

In [ ]:
!nvidia-smi

## PyTorch 버전 확인

In [ ]:
import torch
print(torch.__version__)

## CUDA 버전 확인

In [ ]:
!nvcc -V

# Tensor
PyTorch에서 사용하는 자료구조    
Numpy의 ndarray와 유사    


리스트 형식의 데이터를 사용해서 텐서 생성

In [ ]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)

타입, 모양 확인

슬라이스

내용물 타입, 저장된 장치

Numpy에서 Tensor로 변환 가능 

In [ ]:
import numpy as np
np_array = np.array([[1,2,3],[4,5,6]])
print(np_array)
print(type(np_array))

Tensor를 Numpy로 변환 가능

원하는 속성으로 생성 가능

### Tensor 연산

텐서 2개 생성

In [ ]:
x = torch.tensor([[1,2], [3,4]])
y = torch.tensor([[1,2], [4,8]])
print(x)
print(y)

덧셈, 곱셈, 나눗셈, 행렬곱

연결(concatenate)

모양 변경

In [ ]:
x = torch.rand(4,6)
print(x)
print(x.size())

내용물 확인

## Teosor를 GPU에 할당


GPU 메모리 사용량 확인

In [ ]:
!nvidia-smi

크키가 큰 텐서를 cpu에서 생성

In [ ]:
z = torch.rand(100, 100, 100)
print(z.device)

GPU 메모리 사용량 확인(변화 X)

In [ ]:
!nvidia-smi

cpu에 저장된 z를 gpu에 v로 저장

In [ ]:
device = torch.device('cuda')
v = z.to(device)
print(v.device)

GPU 메모리 사용량 확인(변화 O)

In [ ]:
!nvidia-smi

gpu에 저장된 v를 cpu에 w로 저장 가능

In [ ]:
w = v.to('cpu')
print(w.device)

## torch.autograd
자동 미분 엔진    

### 퍼셉트론 예시
$x_1, x_2$ : 입력값    
$w_1, w_2, b$ : 학습 가능한 파라미터    
$y_t$ : 타겟값    
$y = x_1 w_1 + x_2 w_2 + b$ : 모델의 예측값    
$loss = (y - y_t)^2$ : 손실 함수

In [ ]:
def model(x1, x2, w, b):
    return x1*w[0] + x2*w[1] + b
    
def loss(y_pred,y):
    return (y - y_pred) * (y - y_pred)

데이터 $x_1, x_2, y$와 파라미터 $w_1, w_2, b$ 정의

In [ ]:
x1_data = [1.0, 2.0, 3.0]
x2_data = [2.0, 4.0, 8.0]
yt_data = [3.0, 6.0, 9.0]
w = torch.tensor([1.0, 2.0], requires_grad = True)
w.retain_grad = True
b = torch.tensor([1.0], requires_grad = True)
b.retain_grad = True

초기 모델 성능 확인

In [ ]:
print('target y = {}, predicted y = {}'.format(yt_data[0], model(x1_data[0], x2_data[0], w, b).item()))
print('target y = {}, predicted y = {}'.format(yt_data[1], model(x1_data[1], x2_data[1], w, b).item()))
print('target y = {}, predicted y = {}'.format(yt_data[2], model(x1_data[2], x2_data[2], w, b).item()))

첫번째 데이터에 대한 예측 결과와 타겟    
$x_1 = 1.0, x_2 = 2.0$    
$w_1 = 1.0, w_2 = 2.0, b = 1.0$    
$y = x_1 w_1 + x_2 w_2 + b = 6$        
$y_t = 3.0$

In [ ]:
x1 = x1_data[0]
x2 = x2_data[0]
yt = yt_data[0]
y = model(x1, x2, w, b)
print(y.item())
print(yt)

손실 함수 계산    
$loss = (y - y_t)^2 = 9$

In [ ]:
l = loss(yt, y)
print(l)

#### 역전파
경사 계산 전

In [ ]:
print(w.grad)
print(b.grad)

경사 계산 후    
$l = (y - y_t)^2 = (x_1 w_1 + x_2 w_2 + b - y_t)^2$    
$\frac{\partial l}{\partial w_1} = 2x_1(x_1 w_1 + x_2 w_2 + b - y_t) = 6$    
$\frac{\partial l}{\partial w_2} = 2x_2(x_1 w_1 + x_2 w_2 + b - y_t) = 12$    
$\frac{\partial l}{\partial b} = 2(x_1 w_1 + x_2 w_2 + b - y_t) = 6$    


In [ ]:
l.backward() # 경사 계산
print(w.grad)
print(b.grad)

위의 과정을 그대로 반복할 경우    
$w, b$에 저장된 경사가 남아있어서 이전과 다른 결과가 나옴

In [ ]:
x1 = x1_data[0]
x2 = x2_data[0]
yt = yt_data[0]
y = model(x1, x2, w, b)
l = loss(yt, y)
print(w.grad)
print(b.grad)
l.backward()
print(w.grad)
print(b.grad)

따라서 위에서 계산한 경사를 사용해서 파라미터를 업데이트 한 후에    
저장되어있는 경사를 초기화해줘야 원하는 결과를 얻을 수 있음

In [ ]:
w.grad.data.zero_() # 이전에 계산했던 경사 0으로 초기화
b.grad.data.zero_() # 이전에 계산했던 경사 0으로 초기화
x1 = x1_data[0]
x2 = x2_data[0]
yt = yt_data[0]
y = model(x1, x2, w, b)
l = loss(yt, y)
print(w.grad)
print(b.grad)
l.backward()
print(w.grad)
print(b.grad)

### 모델 학습

learning rate = 0.01    
epochs = 1000    
100번마다 손실 함수 결과 확인

In [ ]:
lr = 0.01
epochs = 1000

for epoch in range(epochs):
    for x1, x2, yt in zip(x1_data, x2_data, yt_data):



    if (epoch + 1) % 100 == 0:
        with torch.no_grad(): # 경사 계산 없이 진행


학습된 모델 성능 확인

In [ ]:
print(w, b)

In [ ]:
print('target y = {}, predicted y = {}'.format(yt_data[0], model(x1_data[0], x2_data[0], w, b).item()))
print('target y = {}, predicted y = {}'.format(yt_data[1], model(x1_data[1], x2_data[1], w, b).item()))
print('target y = {}, predicted y = {}'.format(yt_data[2], model(x1_data[2], x2_data[2], w, b).item()))